# Lab 3
## Motion Capture Clustering

### Team members: Luay Dajani, Dana Geislinger, Chris Morgan, Caroll Rodriguez
##### Github - https://github.com/cdmorgan103/7331DataMiningNoShow

MSDS 7331, 12/02/2018

### Business Understanding 1
#### 10pts
Describe the purpose of the data set you selected (i.e., why was this data collected in the first place?). How will you measure the effectiveness of a good algorithm? Why does your chosen validation method make sense for this specific
dataset and the stakeholders needs?


https://archive.ics.uci.edu/ml/datasets/Motion+Capture+Hand+Postures#

A Vicon motion capture camera system was used to record 12 users performing 5 hand postures with markers attached to a left-handed glove. 

A rigid pattern of markers on the back of the glove was used to establish a local coordinate system for the hand, and 11 other markers were attached to the thumb and fingers of the glove. 3 markers were attached to the thumb with one above the thumbnail and the other two on the knuckles. 2 markers were attached to each finger with one above the fingernail and the other on the joint between the proximal and middle phalanx. 

The 11 markers not part of the rigid pattern were unlabeled; their positions were not explicitly tracked. Consequently, there is no a priori correspondence between the markers of two given records. In addition, due to the resolution of the capture volume and self-occlusion due to the orientation and configuration of the hand and fingers, many records have missing markers. Extraneous markers were also possible due to artifacts in the Vicon software's marker reconstruction/recording process and other objects in the capture volume. As a result, the number of visible markers in a record varied considerably. 

The data presented here is already partially preprocessed. First, all markers were transformed to the local coordinate system of the record containing them. Second, each transformed marker with a norm greater than 200 millimeters was pruned. Finally, any record that contained fewer than 3 markers was removed. The processed data has at most 12 markers per record and at least 3. For more information, see 'Attribute Information'. 

Due to the manner in which data was captured, it is likely that for a given record and user there exists a near duplicate record originating from the same user. We recommend therefore to evaluate classification algorithms on a leave-one-user-out basis wherein each user is iteratively left out from training and used as a test set. One then tests the generalization of the algorithm to new users. A 'User' attribute is provided to accomodate this strategy. 

This dataset may be used for a variety of tasks, the most obvious of which is posture recognition via classification. One may also attempt user identification. Alternatively, one may perform clustering (constrained or unconstrained) to discover marker distributions either as an attempt to predict marker identities or obtain statistical descriptions/visualizations of the postures. 

##### Attribute Information:

Data is provided as a CSV file. A header provides the name of each attribute. An initial dummy record composed entirely of 0s should be ignored. A question mark '?' is used to indicate a missing value. A record corresponds to a single instant or frame as recorded by the camera system. 

'Class' - Integer. The class ID of the given record. Ranges from 1 to 5 with 1=Fist(with thumb out), 2=Stop(hand flat), 3=Point1(point with pointer finger), 4=Point2(point with pointer and middle fingers), 5=Grab(fingers curled as if to grab). 
'User' - Integer. The ID of the user that contributed the record. No meaning other than as an identifier. 
'Xi' - Real. The x-coordinate of the i-th unlabeled marker position. 'i' ranges from 0 to 11. 
'Yi' - Real. The y-coordinate of the i-th unlabeled marker position. 'i' ranges from 0 to 11. 
'Zi' - Real. The z-coordinate of the i-th unlabeled marker position. 'i' ranges from 0 to 11. 

Each record is a set. The i-th marker of a given record does not necessarily correspond to the i-th marker of a different record. One may randomly permute the visible (i.e. not missing) markers of a given record without changing the set that the record represents. For the sake of convenience, all visible markers of a given record are given a lower index than any missing marker. A class is not guaranteed to have even a single record with all markers visible. 


### Data Understanding 1
#### 10pts
Describe the meaning and type of data (scale, values, etc.) for each attribute in the data file. Verify data quality: Are there missing values? Duplicate data? Outliers? Are those mistakes? How do you deal with these problems?

In [1]:
# Import required modules
import numpy as np
import pandas as pd
import math as math
import matplotlib.pyplot as plt
from matplotlib import style
from pprint import pprint
from IPython.display import display

# Load the data into variable 'df'
df = pd.read_csv('https://raw.githubusercontent.com/cdmorgan103/7331DataMiningNoShow/master/LAB3/Postures.csv')

# Get an overview of the raw data
df.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78096 entries, 0 to 78095
Data columns (total 38 columns):
Class    78096 non-null int64
User     78096 non-null int64
X0       78096 non-null float64
Y0       78096 non-null float64
Z0       78096 non-null float64
X1       78096 non-null float64
Y1       78096 non-null float64
Z1       78096 non-null float64
X2       78096 non-null float64
Y2       78096 non-null float64
Z2       78096 non-null float64
X3       78096 non-null object
Y3       78096 non-null object
Z3       78096 non-null object
X4       78096 non-null object
Y4       78096 non-null object
Z4       78096 non-null object
X5       78096 non-null object
Y5       78096 non-null object
Z5       78096 non-null object
X6       78096 non-null object
Y6       78096 non-null object
Z6       78096 non-null object
X7       78096 non-null object
Y7       78096 non-null object
Z7       78096 non-null object
X8       78096 non-null object
Y8       78096 non-null object
Z8       78096 n

In [2]:
# we will remove class 0 
df = df[df.Class !=0 ]

#change ? to none to improve data format
df=df.replace({'?': 'NaN'})

#coerces into numeric
df = df.apply(pd.to_numeric, errors='coerce')

In [3]:
# Get an overview of the raw data
df.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 78095 entries, 1 to 78095
Data columns (total 38 columns):
Class    78095 non-null int64
User     78095 non-null int64
X0       78095 non-null float64
Y0       78095 non-null float64
Z0       78095 non-null float64
X1       78095 non-null float64
Y1       78095 non-null float64
Z1       78095 non-null float64
X2       78095 non-null float64
Y2       78095 non-null float64
Z2       78095 non-null float64
X3       77405 non-null float64
Y3       77405 non-null float64
Z3       77405 non-null float64
X4       74975 non-null float64
Y4       74975 non-null float64
Z4       74975 non-null float64
X5       65072 non-null float64
Y5       65072 non-null float64
Z5       65072 non-null float64
X6       52247 non-null float64
Y6       52247 non-null float64
Z6       52247 non-null float64
X7       38943 non-null float64
Y7       38943 non-null float64
Z7       38943 non-null float64
X8       30563 non-null float64
Y8       30563 non-null float64

Only 31 data points appear for the 11th sensor, and are for one user. We will delete this as a feature as it does not provide any value.

In [4]:
del df['X11']
del df['Y11']
del df['Z11']

   begin work on fix

In [5]:
pd.set_option("display.max_columns",100)

In [6]:
#creates new sub dataframes for x y and z coordinates
dfx=df[['X0','X1','X2','X3','X4','X5','X6','X7','X8','X9','X10']].copy()
dfy=df[['Y0','Y1','Y2','Y3','Y4','Y5','Y6','Y7','Y8','Y9','Y10']].copy()
dfz=df[['Z0','Z1','Z2','Z3','Z4','Z5','Z6','Z7','Z8','Z9','Z10']].copy()

In [7]:
#calculates min value
dfx['xmin']=dfx.min(axis=1)
dfy['ymin']=dfy.min(axis=1)
dfz['zmin']=dfz.min(axis=1)

In [8]:
###takes minimum and subtracts form all
for a in range(11): 
    dfx['X%d' % a] = dfx['X%d' % a] - dfx['xmin']
    
for b in range(11): 
    dfy['Y%d' % b] = dfy['Y%d' % b] - dfy['ymin']
    
for c in range(11): 
    dfz['Z%d' % c] = dfz['Z%d' % c] - dfz['zmin']    

In [9]:
#remove the calculated min values for merging
del dfx['xmin']
del dfy['ymin']
del dfz['zmin']

In [10]:
#create mini dataframe and then concatenate the transformed dataframes
dfmini = df.iloc[:,0:2].copy()
dfnew= pd.concat([dfmini,dfx, dfy,dfz], axis=1)

In [11]:
#reorder the columns
dfnew = dfnew[['Class','User', 'X0','Y0','Z0','X1','Y1','Z1','X2','Y2','Z2','X3','Y3','Z3','X4','Y4','Z4','X5','Y5','Z5',
               'X6','Y6','Z6','X7','Y7','Z7','X8','Y8','Z8','X9','Y9','Z9','X10','Y10','Z10']]

In [ ]:
####alt version

In [ ]:
df['tempx'] = 0-df.X0
df['tempy'] = 0-df.Y0
df['tempz'] = 0-df.Z0

In [ ]:
dfcen=df.copy()

In [ ]:
###x values
dfcen.X0=dfcen.X0+df.tempx
dfcen.X1=dfcen.X1+df.tempx
dfcen.X2=dfcen.X2+df.tempx
dfcen.X3=dfcen.X3+df.tempx
dfcen.X4=dfcen.X4+df.tempx
dfcen.X5=dfcen.X5+df.tempx
dfcen.X6=dfcen.X6+df.tempx
dfcen.X7=dfcen.X7+df.tempx
dfcen.X8=dfcen.X8+df.tempx
dfcen.X9=dfcen.X9+df.tempx
dfcen.X10=dfcen.X10+df.tempx

###y values
dfcen.Y0=dfcen.Y0+df.tempy
dfcen.Y1=dfcen.Y1+df.tempy
dfcen.Y2=dfcen.Y2+df.tempy
dfcen.Y3=dfcen.Y3+df.tempy
dfcen.Y4=dfcen.Y4+df.tempy
dfcen.Y5=dfcen.Y5+df.tempy
dfcen.Y6=dfcen.Y6+df.tempy
dfcen.Y7=dfcen.Y7+df.tempy
dfcen.Y8=dfcen.Y8+df.tempy
dfcen.Y9=dfcen.Y9+df.tempy
dfcen.Y10=dfcen.Y10+df.tempy

###y values
dfcen.Z0=dfcen.Z0+df.tempz
dfcen.Z1=dfcen.Z1+df.tempz
dfcen.Z2=dfcen.Z2+df.tempz
dfcen.Z3=dfcen.Z3+df.tempz
dfcen.Z4=dfcen.Z4+df.tempz
dfcen.Z5=dfcen.Z5+df.tempz
dfcen.Z6=dfcen.Z6+df.tempz
dfcen.Z7=dfcen.Z7+df.tempz
dfcen.Z8=dfcen.Z8+df.tempz
dfcen.Z9=dfcen.Z9+df.tempz
dfcen.Z10=dfcen.Z10+df.tempz


In [ ]:
dfcen.head()

In [ ]:
dfcen.describe()

In [ ]:
dfcen

#### Reformatting the Data for Sensor Clustering

The raw data for this dataset is formatted with each capture being treated as a separate observation. This makes sense if you are classifying the gesture or user, but we are trying to cluster in order to identify which sensor on the glove corresponds to a specific set of coordinates. Therefore, we must stack the data in order to get a new dataset where each observation has only a single X, Y, and Z coordinate, and corresponds to the specific position of an individual sensor module in any given capture. 

The original dataset consists of 78,095 captures with coordinates on up to 11 sensors in any given capture, as well as metadata pertaining to the capture about the user and class the capture was representative of. We will retain this metadata, and also create a new variable that corresponds to the sensor number (0 through 10). However, each coordinate will now be labelled as the raw coordinate number on each cartesian axis (X, Y, Z) regardless of sensor number (as opposed to X0, Y10, Z5, etc. as was the case originally).

At this point, we will now also drop missing datapoints; NaN values in this dataset represent sensors that were obscured or not recorded for some other reason during a capture, and since we are interested in clustering by sensor, it does not further the analysis to include sensor coordinates for which no data was recorded.

In [ ]:
# Create a list of dataframes with X, Y, Z coordinates selected for each individual sensor (0-10)
df_coords = [df.loc[:, ['Class', 'User', 'X%d' % i, 'Y%d' %i, 'Z%d' % i]] for i in range(11)]

for i in range(len(df_coords)):
    # NaN values are dropped at this time
    df_coords[i] = df_coords[i].dropna()
    
    # Coordinate variable labels will now be standardized
    df_coords[i].columns = ['Class', 'User', 'X', 'Y', 'Z']
    
    # Sensor number will now be added as a variable 'sensor' to each dataset
    df_coords[i]['Sensor'] = i
    
# Combine datasets into new dataframe: df2
df2 = pd.concat(df_coords, ignore_index=True)

# Reorder columns with metadata columns first
df2 = df2[['Class', 'User', 'Sensor', 'X', 'Y', 'Z']]

With the newly formatted dataset, we now have 612,209 distinct cartesian coordinates with corresponding Sensor, Class, and User metadata.

In [ ]:
df2.info()

### Data Understanding 2
#### 10pts
Visualize the any important attributes appropriately. Important: Provide an interpretation for any charts or graphs.

##### Number of Datapoints per User by Class (Hand Gesture)

In [ ]:
import pandas as pd
# Print descriptive info for the unique values for each predictor

#1=Fist(with thumb out), 
#2=Stop(hand flat), 
#3=Point1(point with pointer finger), 
#4=Point2(point with pointer and middle fingers), 
#5=Grab(fingers curled as if to grab). 

print('Class:', list(df2.Class.unique()))
print('User:', list(df2.User.unique()))
#df_User = df.groupby(['User','Class'])['Class'].count() 
df_Class = df2.groupby(['Class', 'User'])['User'].count()

df_Class['total'] = sum(df_Class) #df_User 

df_Class

In [ ]:
del df_Class['total']
#plot and format bar chart
plt.figure(figsize=(16,5), dpi=80)
_ = df_Class.plot(kind='bar')
#plt.barh(df_neighborhood, width=0.4, height=0.4, align='center', alpha=0.4)

### Percentage of datapoints for each User by Class (Hand Gesture)

In [ ]:
byClass = df2.groupby(['Class'])['User'].count()
User_byClass = df2.groupby(['Class','User'])['User'].count()

output = User_byClass['Percentage'] = (User_byClass / byClass ) * 100

df_output = output.to_frame("Percentage")
df_output

### Interactive Plotting of User/Class Combinations

In [ ]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode
import pandas as pd
import numpy as np

# Set up plotly API key
plotly.tools.set_credentials_file(username='danageis', api_key='0NAcrzM8YksRAT3uyEZ7')

# Enable offline plots (embed into ipynb)
init_notebook_mode(connected=True)

# Define list of possible users
possible_users = set(df.User.unique())

# Define dictionary of all possible user/class combinations (which ones took pictures of which)
possible_classes = {usr: set(df.where(df.User == usr).Class.dropna().unique()) for usr in possible_users}

# Define functions to create 3D-interactive plots of each user/position combo
def parse_coords(cls=1, usr=1):
    """ Helper function: return all coords for specified User and Class
    
    cls (int): Class (defaults to 1)
    usr (int): User (defaults to 1)
    
    Ret (dict): {dimension (str): coordinates (pd.Series)}
    """
    dims = ['X', 'Y', 'Z']
    fill = lambda suffix: [suffix + str(i) for i in range(1,11)]
    cols = {col: fill(col) for col in dims}
    coords = {dim: df[cols[dim]].where(df.User == usr).where(df.Class == cls).stack() for dim in dims}
    return coords

def plot_3d(cls=1, usr=1, title=None, allpts=False, clusters=np.array([0])):
    """ Return 3D interactive plot for given coordinates set (single user/sign combos)
    
    cls (int): Class (defaults to 1)
    usr (int): User (defaults to 1)
    title (str): Title of the plot (defaults to "User {usr} Position {cls}")
    allpts (bool): Whether or not this is meant to cluster all points in the dataset (df2)
        All 600k+ points is too many to plot interactively; this will sample 20,000 points randomly and graph them
    clusters (np.array): Clusters to use for color-coding (only valid for allpts at this time)
    
    Ret (plotly.graph_objs.Scatter3d figure object)
    """
    
    # Make sure user/class designation is valid
    if not usr in possible_users:
        return "ERROR: User %d does not have any data. Try again with a different user specified." % usr
    elif not cls in possible_classes[usr]:
        return "ERROR: User %d does not have any data for Class %d. Try again with a different class specified." % (usr, cls)
    
    # Process points to draw
    if allpts:
        # Sample 20,000 random coords w/o replacement
        coords = df2.copy()
        if clusters.any():
            coords['cluster'] = clusters
        coords = coords.sample(n=20000, replace=False)
    else:
        coords = parse_coords(cls, usr)
        
    # Name plot if title not provided (default behavior)
    if not title:
        if allpts:
            title = "20,000 Random Points from Dataset"
            if clusters.any():
                title += " (Clustered)"
        else:
            title = "User %d Position %d" % (usr, cls)

    # Color-code clustered points, if provided
    if allpts and clusters.any():
        marker_params = dict(size=5,
                             color=coords.cluster,
                             colorscale='Portland'
                             )
    else:
        marker_params = dict(size=5)
            
    data = [go.Scatter3d(x=coords['X'],
                         y=coords['Y'],
                         z=coords['Z'],
                         mode='markers',
                         marker=marker_params
                         )
            ]

    layout = dict(width=800,
                  height=700,
                  autosize=True,
                  title=title,
                  scene=dict(xaxis=dict(gridcolor='rgb(255, 255, 255)',
                                        zerolinecolor='rgb(255, 255, 255)',
                                        showbackground=True,
                                        backgroundcolor='rgb(230, 230,230)'
                                        ),
                             yaxis=dict(gridcolor='rgb(255, 255, 255)',
                                        zerolinecolor='rgb(255, 255, 255)',
                                        showbackground=True,
                                        backgroundcolor='rgb(230, 230,230)'
                                        ),
                             zaxis=dict(gridcolor='rgb(255, 255, 255)',
                                        zerolinecolor='rgb(255, 255, 255)',
                                        showbackground=True,
                                        backgroundcolor='rgb(230, 230,230)'
                                        ),
                             camera=dict(up=dict(x=0,
                                                 y=0,
                                                 z=1
                                                 ),
                                         eye=dict(x=-1.7428,
                                                  y=1.0707,
                                                  z=0.7100,
                                                  )
                                         ),
                             aspectratio = dict(x=1,
                                                y=1,
                                                z=0.7
                                                ),
                             aspectmode = 'manual'
                             ),
                )

    fig = dict(data=data, layout=layout)

    return py.iplot(fig, filename="Hands-%s" % title)

In [ ]:
from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
plot_3d(cls=1, usr=1)

### Modeling and Evaluation 1
#### 10 pts
Train and adjust parameters

For this data, we will be attempting to cluster the data on the basis of the sensors used to collect the data. As described in the business understanding section, this dataset consists of snapshot measurements taken from a special glove fitted with sensors to track a person's hand movements and map them to 3-dimensional space. The dataset has recorded data for up to 11 sensors in each capture (some may have been obscured when measurement was performed), and associated 3D coordinates for each sensor. Captures were taken for 13 users who were performing one of five pre-defined gestures with their hand.

Our goal is to cluster each data point based on which of the 11 possible sensors it corresponds to. Each record in the dataset has been formatted so that it represents a single coordinate location for an individual sensor. To accomplish this we will attempt clustering with 11 total clusters defined in our clustering algorithms, ideally corresponding with each of the sensors that is present on the glove. We will begin by performing cluster analysis of all points aggregated together:

In [ ]:
plot_3d(allpts=True)

In [ ]:
plot_3d(allpts=True)

Shown above are 2 random samplings of 20,000 points each from the dataset. This was done to illustrate that when multiple random samples are plotted in 3-dimensional space, the distributions are different each time but there are similar patterns that can be observed consistently such as 2 large elliptoid clusters on the right side and some smaller clusters on the left side. This provides some anecdotal evidence that clustering may be successful using all available data aggregated from every user and every gesture class.

### GMM Clustering

Visualizing the data set as we have in the two random samples above, it seems that while there are some obvious discrete clusters in 3-dimensional space these clusters are not necessarily uniform globular clusters. These type of clusters might be well defined using Gaussian mixture model clustering which would allow for normally distributed clusters of varying shapes and sizes. We will first attempt to fit a Gaussian mixture model to obtain clusters for the 11 sensors in the dataset.

In [ ]:
from sklearn.mixture import GaussianMixture as GMM

# Fit 11 clusters
#  Set random state to '123456789' to make sure same result achieved each time
gmm = GMM(n_components=11,
          random_state=123456789
          )
gmm_X = df2[['X','Y','Z']]
gmm.fit(gmm_X)
gmm_clusters = gmm.predict(gmm_X)

In [ ]:
from mpl_toolkits import mplot3d
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
ax = plt.axes(projection='3d')

# Data for a three-dimensional line
zline = np.linspace(0, 15, 1000)
xline = np.sin(zline)
yline = np.cos(zline)
ax.plot3D(xline, yline, zline, 'gray')

# Data for three-dimensional scattered points
zdata = gmm_X['Z'].astype(float)
xdata = gmm_X['X'].astype(float)
ydata = gmm_X['Y'].astype(float)
_ = ax.scatter3D(gmm_X['X'], gmm_X['Y'], gmm_X['Z'], c=gmm_clusters)

In [ ]:
plot_3d(allpts=True, clusters=gmm_clusters)

We plot the basic GMM clusters above, both as a static image consisting of all points (more than 600,000) and an interactive plot that is sampled as 20,000 random points from the dataset color-coded by cluster. From these visuals, we can see that there does appear to be decent clustering.

From the data description, we know that there are 2 sensors on each finger and 3 sensors on the thumb. We also know that there are several positions in which multiple (sometimes more than half) of the sensors are obscured from view and not recorded. Therefore, this clustering makes sense from a logical perspective, but it could likely still be improved.

Another interesting comparison is the distribution of clusters as defined by GMM. While we do not know from the raw data which sensors correspond to which point on the glove, we do know that in a majority of recorded instances there are sensors that were unable to be recorded. It would be interesting to note if this clustering algorithm created clusters of varying sizes, which might be indicative of an uneven distribution of which sensors were not recorded most or least often.

In [ ]:
# Create a new dataframe with the clusters defined as a new variable
df_gmm = gmm_X
df_gmm['Clusters'] = gmm_clusters

# Count the number of instances in each cluster and convert to a percentage of the total
cluster_pcts = df_gmm.groupby('Clusters').count()['X'] / len(gmm_X) * 100

# Plot the distribution
ax = cluster_pcts.plot(kind='bar', title="Percentage of Each GMM-Defined Cluster")
_ = ax.set_ylabel("Percentage of Total Points (%)")

It is interesting to note from this distribution that there are significantly varying distributions between clusters. If these clusters do in fact coincide with sensors on the glove, this could provide evidence that certain sensors appear more often than others.

This also makes logical sense, since it would be expected that some of the gestures being performed would have obscured at least part of the hand. The distribution shows that from GMM, there are 2 clusters defined that are very scarce in the total (both less than 5%) compared to the rest. This could be accounted for as the 2 sensors present on a finger that is often obscured, such as the pinky finger. Conversely, there are 2 clusters that stand out as about the same percentage as eachother (almost 14%) but both significantly higher than most of the other clusters. These cluster could represent sensors on fingers that are needed to perform most of the gestures recorded, so they might show up regardless of user or class.

### Modeling and Evaluation 2
#### 10pts
Evaluate and Compare

#####


In [ ]:
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN


data = np.array(df2.to_array())

n_clusters = 11

model = KMeans(init='k-means++', n_clusters=n_clusters, n_init=1).fit(data)
# model = AgglomerativeClustering(linkage='ward', n_clusters=n_clusters).fit(data)
model = DBSCAN(eps=0.15, min_samples=10).fit(data)

labels = model.labels_
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

print('Estimated number of clusters: %d' % n_clusters_)
print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
print("Adjusted Rand Index: %0.3f"
      % metrics.adjusted_rand_score(labels_true, labels))
print("Adjusted Mutual Information: %0.3f"
      % metrics.adjusted_mutual_info_score(labels_true, labels))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(data, labels))

silhouette_per_sample = metrics.silhouette_samples(data,labels)

# get the middle of each cluster
centroids = []
for lab in range(0,n_clusters_):
    centroids.append( [np.mean(data[np.where(labels==lab),0]), np.mean(data[np.where(labels==lab),1])] )
centroids = np.array(centroids)

plt.figure()
plt.scatter(data[:, 0], data[:, 1], c=labels,
                    cmap=plt.cm.spectral, s=5, linewidths=0)

if len(centroids)>0:
    plt.scatter(centroids[:, 0], centroids[:, 1],
                marker='+', s=200, linewidths=3, color='k', zorder=10)  # plot the centroids
plt.title('Estimated number of clusters: %d' % n_clusters_)

plt.figure()
plt.scatter(data[:, 0], data[:, 1], c=silhouette_per_sample,
                cmap=plt.cm.gray, s=5, linewidths=0)
plt.title('Clusters with silhouette coefficient coloring')
plt.show()

In [ ]:
# do it for the k-means
from sklearn.cluster import KMeans
data = df2
seuclid = []
scosine = []
k = range(2,10)
for i in k:
    kmeans_model = KMeans(n_clusters=i, init="k-means++").fit(data)
    labels = kmeans_model.labels_
    seuclid.append(metrics.silhouette_score(data, labels, metric='euclidean'))
    scosine.append(metrics.silhouette_score(data, labels, metric='cosine'))
    
plt.figure(figsize=(10,5))
plt.plot(k,seuclid,label='euclidean')
plt.plot(k,scosine,label='cosine')
plt.ylabel("Silhouette")
plt.xlabel("Cluster")
plt.title("Silhouette vs Cluster Size")
plt.legend()
plt.show()

### Modeling and Evaluation 3
#### 10 pts
Visualize Results

### Modeling and Evaluation 4
#### 20 pts
Summarize the Ramifications

### Deployment 
#### 10 pts
Be critical of your performance and tell the reader how you current model might be usable by other parties. Did you achieve your goals? If not, can you reign in the utility of your modeling? How useful is your model for interested parties (i.e., the companies or organizations that might want to use it)? How would your deploy your model for interested parties? What other data should be collected? How often would the model need to be updated, etc.?

### Exceptional Work
#### 10 pts
You have free reign to provide additional analyses or combine analyses.

In [ ]:
from mpl_toolkits import mplot3d

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#1=Fist(with thumb out), 
#2=Stop(hand flat), 
#3=Point1(point with pointer finger), 
#4=Point2(point with pointer and middle fingers), 
#5=Grab(fingers curled as if to grab). 

#filter certain users for plotting below
df_user6 = df[df.User == 6][df.Class == 2]
df_user14 = df[df.User == 14][df.Class == 1]
df_user5 = df[df.User == 5][df.Class == 4]
df_user8 = df[df.User == 8][df.Class == 3]

#One row in data table - represnts on set of data points
df_row48977 = df[48976:48977][df.Class == 5]

# feature categories
X_features = ['X0','X1','X2','X3','X4','X5','X6','X7','X8','X9','X10']
Y_features = ['Y0','Y1','Y2','Y3','Y4','Y5','Y6','Y7','Y8','Y9','Y10']
Z_features = ['Z0','Z1','Z2','Z3','Z4','Z5','Z6','Z7','Z8','Z9','Z10']

##### One record plotted in 3D show Class 5 (Grab Gestures)

In [ ]:
#one set of markers
ax = plt.axes(projection='3d')

# Data for a three-dimensional line
zline = np.linspace(0, 15, 1000)
xline = np.sin(zline)
yline = np.cos(zline)
ax.plot3D(xline, yline, zline, 'gray')

# Data for three-dimensional scattered points
zdata = df_row48977[Z_features].astype(float)
xdata = df_row48977[X_features].astype(float)
ydata = df_row48977[Y_features].astype(float)
ax.scatter3D(xdata, ydata, zdata);

##### All User 6's records plotted in 3D with Posture 3 (Stop sign)

In [ ]:
ax = plt.axes(projection='3d')

# Data for a three-dimensional line
zline = np.linspace(0, 15, 1000)
xline = np.sin(zline)
yline = np.cos(zline)
ax.plot3D(xline, yline, zline, 'gray')

# Data for three-dimensional scattered points
zdata = df_user6[Z_features].astype(float)
xdata = df_user6[X_features].astype(float)
ydata = df_user6[Y_features].astype(float)
ax.scatter3D(xdata, ydata, zdata);

##### All User 14's records plotted in 3D with Class 1 (Fist Gesture)

In [ ]:
ax = plt.axes(projection='3d')

# Data for a three-dimensional line
zline = np.linspace(0, 15, 1000)
xline = np.sin(zline)
yline = np.cos(zline)
ax.plot3D(xline, yline, zline, 'gray')

# Data for three-dimensional scattered points
zdata = df_user14[Z_features].astype(float)
xdata = df_user14[X_features].astype(float)
ydata = df_user14[Y_features].astype(float)
ax.scatter3D(xdata, ydata, zdata);

##### All User 8's records plotted in 3D with Class 3 (point with pointer finger Gesture)

In [ ]:
ax = plt.axes(projection='3d')

# Data for a three-dimensional line
zline = np.linspace(0, 15, 1000)
xline = np.sin(zline)
yline = np.cos(zline)
ax.plot3D(xline, yline, zline, 'gray')

# Data for three-dimensional scattered points
zdata = df_user8[Z_features].astype(float)
xdata = df_user8[X_features].astype(float)
ydata = df_user8[Y_features].astype(float)
ax.scatter3D(xdata, ydata, zdata);

##### All User 5's records plotted in 3D with Class 5 (Grab Gesture)

In [ ]:
ax = plt.axes(projection='3d')

# Data for a three-dimensional line
zline = np.linspace(0, 15, 1000)
xline = np.sin(zline)
yline = np.cos(zline)
ax.plot3D(xline, yline, zline, 'gray')

# Data for three-dimensional scattered points
zdata = df_user5[Z_features].astype(float)
xdata = df_user5[X_features].astype(float)
ydata = df_user5[Y_features].astype(float)
ax.scatter3D(xdata, ydata, zdata);

##### All User 5's records plotted in 3D in a Wireframe show Class 2 (Stop Gesture)

In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot_wireframe(df_user6[X_features], df_user6[Y_features], df_user6[Z_features], color='black')
ax.set_title('wireframe');
